In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from rolling_ta.data import CSVLoader
from rolling_ta.volume import NumbaOBV
from rolling_ta.logging import logger
from ta.volume import OnBalanceVolumeIndicator
from time import time

DEBUG - Cache [numba=True]


In [3]:
xls_loader = CSVLoader()
data = xls_loader.read_resource().copy()
data.info()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  10000 non-null  float64
 1   open       10000 non-null  float64
 2   high       10000 non-null  float64
 3   low        10000 non-null  float64
 4   close      10000 non-null  float64
 5   volume     10000 non-null  float64
dtypes: float64(6)
memory usage: 468.9 KB


In [4]:
%%timeit
obv = OnBalanceVolumeIndicator(data["close"], data["volume"])
obv.on_balance_volume()

272 μs ± 8.19 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
%%timeit
NumbaOBV(data, memory=False)

51.5 μs ± 819 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
obv = NumbaOBV(data.iloc[:9999])

In [7]:
%%timeit
obv.update(data.iloc[9999])

19.7 μs ± 536 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
